In [ ]:
import pandas as pd

# import cleaned versions of files

# orderlines_cl.csv
url = 'https://drive.google.com/file/d/1lrSr_vVbXN7QSVSScf68DpWwvCMclK5m/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
orderlines = pd.read_csv(path)

# orders_cl.csv
url = 'https://drive.google.com/file/d/1cGjJ9o3vtwjK0Sohyr3YVTccXfnTPijT/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
orders = pd.read_csv(path)

# brands_cl.csv
url = 'https://drive.google.com/file/d/1XGyabaa4mAkjixMk3XPgx_14OoSse3rs/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
brands = pd.read_csv(path)

# products_cl.csv
url = 'https://drive.google.com/file/d/1rHC8M-HG13FtVncImXBydgDtIHyCPH0g/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
products = pd.read_csv(path)

# Things to check: 

* [x] take a look at dates, do they match between orders and orderlines.
* [_] info between orders and orderlines: 
    * [x] do we have the same orders on both df?
    * [_] revenue? Check if prices between products and orderlines matches.

In [ ]:
orders.sample(5)

,order_id,created_date,total_paid,state,check_orders
157125,473650,2017-12-27 17:19:46,186.99000,Shopping Basket,True
143144,457988,2017-12-09 01:37:31,3653.58000,Shopping Basket,True
136238,450617,2017-11-28 15:48:54,124.99000,Pending,True
33720,336223,2017-03-17 13:00:27,89.98000,Pending,True
97352,406598,2017-09-30 16:01:41,1180.81000,Completed,True


In [ ]:
orderlines.sample(5)

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
152773,1421649,430637,0,1,WAC0216,1699.66000,2017-11-22 13:38:59,1699.66000,True
219786,1530432,478718,0,1,PHI0084,164.99000,2018-01-01 20:08:14,164.99000,True
163231,1440035,434595,0,3,DLL0055,329.00000,2017-11-24 17:48:39,987.00000,True
174223,1458305,434804,0,1,APP2135,1212.63000,2017-11-27 14:22:44,1212.63000,True
37877,1199420,332703,0,1,TUC0298,29.99000,2017-03-06 18:39:50,29.99000,True


In [ ]:
# check if order id on orders exist on orderlines
(
pd.DataFrame(orders["order_id"])
    .rename(columns={"order_id":"id_order"})
    .assign(is_in_orderlines = lambda x: x["id_order"].isin(orderlines["id_order"]))
    .query("is_in_orderlines")
).shape[0]

204691

In [ ]:
# check of order id on orderlines exists on orders
(
pd.DataFrame(orderlines["id_order"])
    .rename(columns={"id_order":"order_id"})
    .assign(is_in_orders = lambda x: x["order_id"].isin(orders["order_id"]))
    .query("is_in_orders")
).shape[0]

293743

In [ ]:
orders.head()

,order_id,created_date,total_paid,state,check_orders
0,241319,2017-01-02 13:35:40,44.99000,Cancelled,True
1,241423,2017-11-06 13:10:02,136.15000,Completed,True
2,242832,2017-12-31 17:40:03,15.76000,Completed,True
3,243330,2017-02-16 10:59:38,84.98000,Completed,True
4,243784,2017-11-24 13:35:19,157.86000,Cancelled,True


In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
(
orderlines
    .groupby(["id_order"])
    .agg(total_price = ("total_price","sum"))
    .reset_index()
    .merge(orders, how="inner", left_on = "id_order", right_on = "order_id")
    .filter(["id_order","total_price","total_paid"])
    .assign(diff = lambda x: x["total_paid"] - x["total_price"])
    .query("diff < 0.02 & diff > 0")
    .sum()
    # .query("0 < diff < 30")
    # ["diff"].hist(bins=50)
    # .sample(10)
)


id_order      3885483363.00000
total_price     16073735.01000
total_paid      16073799.13000
diff                  64.12000
dtype: float64

### Explanation how to use `DataFrame.merge()`

In [ ]:
df_1 = pd.DataFrame({
    "id":["Hesham","Carolina","Vasil"],
    "x":[1, 2, 3]
})
df_1

,id,x
0,Hesham,1
1,Carolina,2
2,Vasil,3


In [ ]:
df_2 = pd.DataFrame({
    "another_id":["Hesham","Carolina"],
    "color":["blue","red"]
})
df_2

,another_id,color
0,Hesham,blue
1,Carolina,red


In [ ]:
df_1.merge(df_2, left_on="id", right_on="another_id", how="outer")

,id,x,another_id,color
0,Hesham,1,Hesham,blue
1,Carolina,2,Carolina,red
2,Vasil,3,NaN,NaN


Create discounts columns on orderlines: 

In [ ]:
(
orderlines
    .filter(["id_order","sku","unit_price"])
    .merge(products.filter(["sku","desc","max_price_orderlines"]), on="sku", how="left")
    .assign(disc = lambda x: x["max_price_orderlines"] - x["unit_price"])
    .dropna()
    .sample(20)
)

,id_order,sku,unit_price,desc,max_price_orderlines,disc
177128,450282,REP0309,59.99000,Repair service including parts and labor for i...,59.99000,0.00000
70853,360666,SEV0028,19.99000,diagnosis to repair MacBook Pro.,108.90000,88.91000
285089,521932,REP0365,69.99000,Repair service including parts and labor for i...,69.99000,0.00000
221953,480666,FIB0011,59.99000,embedded module to control your consumption ou...,59.99000,0.00000
125016,407481,REP0391,69.99000,Repair service including parts and labor for i...,69.99000,0.00000
152791,431801,SEV0024,59.99000,installation RAM HDD + SSD + on your Mac mini ...,59.99000,0.00000
169898,437533,CRU0015-2,135.14000,RAM 16GB (2x8GB) 135V MacBook Pro iMac (2012/2...,177.99000,42.85000
2946,302265,IOT0018,24.99000,Magnetic car holder compatible with all models...,24.99000,0.00000
223183,481774,REP0274,59.99000,Repair service including parts and labor for i...,59.99000,0.00000
136901,417932,CRU0015-2,148.99000,RAM 16GB (2x8GB) 135V MacBook Pro iMac (2012/2...,177.99000,29.00000
